# Hello CUDA-Q with Braket Hybrid Jobs
[CUDA-Q](https://nvidia.github.io/cuda-quantum/latest/index.html) offers a unified programming model designed for hybrid workloads that run on CPUs, GPUs and QPUs. In this notebook, you will learn how to run CUDA-Q programs on Amazon Braket. Specifically, we will be using the Bring Your Own Container (BYOC) feature of Braket Hybrid Jobs. BYOC enables you to configure the environment that you want to use in your jobs. This notebook assumes basic knowledge of Amazon Braket Hybrid Jobs. You can learn about Hybrid Jobs from [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html) in the Amazon Braket Developer Guide and the notebooks in [Amazon Braket Examples](https://github.com/amazon-braket/amazon-braket-examples/tree/main/examples/hybrid_jobs).

The shell script, `container_build_and_push.sh`, in the "container" folder will build a Docker container and upload the container image to your [ECR](https://aws.amazon.com/ecr/) repository with the name and region that you specify. You run the script by providing a image name and AWS region where to store the image:
```
container/container_build_and_push.sh <image-name> <region-name>
```
The container image will persist in your ECR repository until you update it. The ARN of the image will take the format: `<aws-account-id>.dkr.ecr.<region-name>.amazonaws.com/<container-image-name>:latest`. To learn more about what's under the hood when you call the shell script and how to configure the environment of the job container, you can view the [Appendix](#Appendix:-Container-build-procedure) of this notebook.

In [ ]:
!container/container_build_and_push.sh amazon-braket-cudaq-job us-west-2

## Running CUDA-Q with Braket
Now, we have prepared the environment for CUDA-Q in a container image. Let's run our first CUDA-Q job! First, we start with the necessary imports.

In [ ]:
import numpy as np
from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn

We also prepare the URI of the container image. Fill the proper value of `aws_account_id`, `region_name` and `container_image_name` in the cell below. For example, with the shell command above, `region_name="us-west-2"` and `container_image_name="amazon-braket-cudaq-job"`. The cell below prints out the image URI. When you use a container image to run a job, it ensures that your code is run in the same environment every time. 

In [ ]:
aws_account_id = "<aws-account-id>"
region_name = "<region-name>"
container_image_name = "<container-image-name>"

image_uri = f"{aws_account_id}.dkr.ecr.{region_name}.amazonaws.com/{container_image_name}:latest"
print(image_uri)

## Test your CUDA-Q job locally
Before submitting a job, it is recommended to test with a local job. A local job runs scripts in a container locally. It is a good way to test your code with a small problem size before scaling up. Note, running a local hybrid job requires Docker installed on the local computer. 

Here, let's use a Bell circuit with CUDA-Q for the local job. This example does not require a GPU to run. The `hello_quantum` function in the code snippet below defines an experiment to sample a Bell circuit. The string `qpp-cpu` in the `device` keyword argument of the decorator is the name of a CUDA-Q CPU simulator. You can view the tutorial of CUDA-Q and the available backends in the [CUDA-Q documentation](https://nvidia.github.io/cuda-quantum/latest/index.html).

When called, the decorated `hello_quantum` function starts a local job because of the keyword `local=True` in the `hybrid_job` decorator. The code inside the `hello_quantum` function will run locally in the environment defined by the container image that you have built earlier.

In [ ]:
@hybrid_job(device="local:nvidia/qpp-cpu", image_uri=image_uri, local=True)
def hello_quantum():
    import cudaq

    # define the backend
    device = get_job_device_arn()
    cudaq.set_target(device.split("/")[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # define the Bell circuit
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])

    # sample the Bell circuit
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print("Samples: ", measurement_probabilities)

    return measurement_probabilities

Let's test your CUDA-Q job!

In [ ]:
hello_quantum()

## Run your CUDA-Q job
After testing locally that your code works correctly in the container environment, you can remove the `local=True` keyword argument (or, equivalently, set `local=False`), so the next job will run on an AWS-managed compute instance. This is great to scale up to larger instances or parallelize your workload over multiple instances.

In [ ]:
@hybrid_job(device="local:nvidia/qpp-cpu", image_uri=image_uri)
def hello_quantum():
    import cudaq

    device = get_job_device_arn()
    cudaq.set_target(device.split("/")[-1])
    print(cudaq.get_target())

    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])

    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print("Samples: ", measurement_probabilities)

    return measurement_probabilities


job = hello_quantum()
print(job.arn)

When called, the decorated `hello_quantum` function will create a Braket hybrid job on AWS, running the code defined in the decorated function with the environment specified by the container image built above. You can view the progress of your job with `job.state()` or in the "Hybrid jobs" tab of the Amazon Braket Console.

In [ ]:
result = job.result()
print(result)

## Summary
This notebook shows you how to run your first CUDA-Q program with Amazon Braket Hybrid Jobs. Using the BYOC feature of Amazon Braket and a shell script we provide, you can register a CUDA-Q environment with a few lines of code. Once you have registered your CUDA-Q container image, you can run CUDA-Q programs with Braket Hybrid Jobs and scale your workloads up and out with the range of compute options provided by AWS. In the next tutorials, we will show you how to run CUDA-Q simulations on GPUs ([notebook](1_simulation_with_GPUs.ipynb)) and distribute workloads across multiple instances ([notebook](2_parallel_simulations.ipynb)).

## Appendix: Procedure of the container build

When the shell script, `container_build_and_push.sh`, is called, a Docker container is built with CUDA-Q and other GPU related settings configured. The procedure for BYOC is presented in this page from [Braker Developer Guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-byoc.html). The required files for building a container with CUDA-Q are in the "container" folder, including
- `Dockerfile`: Describes how the container is built.
- `requirements.txt`: Additional Python dependencies to include.
- `braket_container.py`: The start-up script of a job container.

These files include basic settings to use CUDA-Q in jobs. You can modify these files to suit your needs. For example, you can add more Python dependencies to `requirements.txt` and other dependencies to `Dockerfile`.

The shell script `container_build_and_push.sh` automates the procedure of building on top of a Braket managed container and pushing your custom container image. The shell script requires Docker to be installed on your local machine. IF you are using Braket NBI, Docker is already installed. If you don't have Docker in your local environment, you can follow the [instruction in this page](https://www.docker.com/) to install it. The shell script takes two parameters:
- image-name
- region-name

The shell script can be called with the following syntax:
```
container/container_build_and_push.sh <image-name> <region-name>
```
By default, the permission policy `AmazonBraketFullAccess` grants access to ECR container images which name start with `amazon-braket`. If you have name the container image differently, you may can modify the perimission of the role you are using. You can view [this page](https://docs.aws.amazon.com/AmazonECR/latest/userguide/repository-policies.html) about permission policy in ECR.